[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/building_model.ipynb)

# Imports 

In [0]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [2]:
!pip install -U bcolz

    100% |████████████████████████████████| 1.5MB 10.2MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [3]:
!pip install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 7.2MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

# GPU Settings 

In [5]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

Using gpu: False 


# Data Loader

In [0]:
# loader taken from class example: parameters to verify

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # statistics from imagenet ? 

prep1 = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
data_dir = '/content/data/sample'
#data_dir = '/content/data'

batch_size = 1
#batch_size = 64

In [0]:
# dictionary of datasets - in this case only train data set 
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train']}

In [9]:
print(dsets['train'].imgs[:5]) # 5 first images and labels 
print(dsets['train'].imgs[len(dsets['train'])-5:]) # 5 last images and labels 

[('/content/data/sample/train/clear/train_1.jpg', 0), ('/content/data/sample/train/clear/train_10.jpg', 0), ('/content/data/sample/train/clear/train_11.jpg', 0), ('/content/data/sample/train/clear/train_13.jpg', 0), ('/content/data/sample/train/clear/train_15.jpg', 0)]
[('/content/data/sample/train/cloudy/train_68.jpg', 1), ('/content/data/sample/train/cloudy/train_72.jpg', 1), ('/content/data/sample/train/cloudy/train_77.jpg', 1), ('/content/data/sample/train/cloudy/train_87.jpg', 1), ('/content/data/sample/train/cloudy/train_97.jpg', 1)]


In [10]:
dset_classes = dsets['train'].classes
dset_classes # binary classes to begin 

['clear', 'cloudy']

In [0]:
# dictionary of data loaders - again only train for now 
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=False, num_workers=0)
                for x in ['train']}

# Model: simple classifier from scratch

In [0]:
# need to understand parameters in more details !!!!

import torch.nn as nn
import torch.nn.functional as F # other way to define layers: when no need to update parameters 

# max pooling layer is not in init: no parameters to update ie. deterministic function  

class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__() # do the init from the inherited class
        # then what are the parameters of the network: need to be defined in init 
        # fill the missing entries below
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1) # check meaning of arguments  
        self.fc = nn.Linear(in_features=32*32*64, out_features=2)
        
    def forward(self,x):
        # implement your network here, use F.max_pool2d, F.log_softmax and do not forget to flatten your vector
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=7, stride=7)
        x = x.view(-1, 32*32*64) # flatten 
        x = self.fc(x)
        return F.log_softmax(x, dim=1) # softmax across the line ! not the component 
    
# since we inherit from nn module, the forward is called by default (do not call classifier.forward() !!)
    
use_gpu = torch.cuda.is_available()
if use_gpu:
    classifier = classifier.cuda()

# Model using vgg16

In [0]:
"""model_vgg = models.vgg16()

for param in model_vgg.parameters():
    param.requires_grad = False
model_vgg.classifier._modules['6'] = nn.Linear(4096, 2)
#model_vgg.classifier[6].out_features = 2
#for param in model_vgg.classifier[6].parameters():
#    param.requires_grad = True

if use_gpu:
    model_vgg = model_vgg.cuda()"""

# Training Framework

In [0]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()    
                
            # batch_size
            bs = labels.size(0)
            
            # define the loss to minimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            # define the optimizer
            optimizer = optimizer
            optimizer.zero_grad()
            # backpropagation
            loss.backward()
            optimizer.step()
            
            # predictions to get statistics 
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == labels.data)
            size += bs
        # epoch statistics     
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

# Running training epochs

In [15]:
# using the model from scratch

# instanciate the model 
conv_class = classifier()

# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_cl = torch.optim.SGD(conv_class.parameters(), lr=learning_rate)


# and train for 10 epochs
l_t, a_t = train(conv_class,dset_loaders['train'],loss_fn,optimizer_cl,n_epochs = 10)

Train - Loss: 0.5180 Acc: 0.8750
Train - Loss: 0.7340 Acc: 0.8250
Train - Loss: 0.5505 Acc: 0.8250
Train - Loss: 1.0344 Acc: 0.9000
Train - Loss: 0.2715 Acc: 0.8750
Train - Loss: 0.1529 Acc: 0.9250
Train - Loss: 0.0946 Acc: 0.9500
Train - Loss: 0.0758 Acc: 0.9750
Train - Loss: 0.0629 Acc: 1.0000
Train - Loss: 0.0536 Acc: 1.0000


In [16]:
# using vgg16 

"""loss_fn = nn.CrossEntropyLoss()
lr = 0.01
optimizer_vgg = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = lr)

train(model_vgg.classifier, dset_loaders['train'], loss_fn, n_epochs=10, optimizer=optimizer_vgg)"""

"loss_fn = nn.CrossEntropyLoss()\nlr = 0.01\noptimizer_vgg = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = lr)\n\ntrain(model_vgg.classifier, dset_loaders['train'], loss_fn, n_epochs=10, optimizer=optimizer_vgg)"